In [1]:
import pandas as pd

df=pd.read_csv(r"F:\MOOO\AI_AMIT_DEPI\projects\ML project\archive\T.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [3]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [4]:
df.drop(["Loan_ID"],axis=1,inplace=True)

In [5]:
df["TotalIncome"]=df["ApplicantIncome"]+df["CoapplicantIncome"]
df.drop(["ApplicantIncome","CoapplicantIncome"],axis=1,inplace=True)

In [6]:
df

,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome
0,Male,No,0,Graduate,No,NaN,360.0,1.0,Urban,Y,5849.0
1,Male,Yes,1,Graduate,No,128.0,360.0,1.0,Rural,N,6091.0
2,Male,Yes,0,Graduate,Yes,66.0,360.0,1.0,Urban,Y,3000.0
3,Male,Yes,0,Not Graduate,No,120.0,360.0,1.0,Urban,Y,4941.0
4,Male,No,0,Graduate,No,141.0,360.0,1.0,Urban,Y,6000.0
...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,71.0,360.0,1.0,Rural,Y,2900.0
610,Male,Yes,3+,Graduate,No,40.0,180.0,1.0,Rural,Y,4106.0
611,Male,Yes,1,Graduate,No,253.0,360.0,1.0,Urban,Y,8312.0
612,Male,Yes,2,Graduate,No,187.0,360.0,1.0,Urban,Y,7583.0


In [7]:
from datasist.structdata import detect_outliers
dirty_cols =df.select_dtypes(['int64','float64'])
dirty_idx=detect_outliers(data=df,n=0,features=dirty_cols)
df.drop(dirty_idx,inplace=True)

In [8]:
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy='mean')
dirty_cols=dirty_cols.columns
df[dirty_cols]=imputer.fit_transform(df[dirty_cols])

In [9]:
imputer=SimpleImputer(strategy="most_frequent")
dirty_cols=df.select_dtypes(['object'])
dirty_cols=dirty_cols.columns
df[dirty_cols]=imputer.fit_transform(df[dirty_cols])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 564 entries, 0 to 613
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Gender            564 non-null    object 
 1   Married           564 non-null    object 
 2   Dependents        564 non-null    object 
 3   Education         564 non-null    object 
 4   Self_Employed     564 non-null    object 
 5   LoanAmount        564 non-null    float64
 6   Loan_Amount_Term  564 non-null    float64
 7   Credit_History    564 non-null    float64
 8   Property_Area     564 non-null    object 
 9   Loan_Status       564 non-null    object 
 10  TotalIncome       564 non-null    float64
dtypes: float64(4), object(7)
memory usage: 52.9+ KB


In [11]:
df = pd.get_dummies(df, columns=["Gender","Married","Education","Self_Employed","Loan_Status","Dependents","Property_Area"], drop_first=True)

In [12]:
from sklearn.model_selection import train_test_split
y=df["Loan_Status_Y"]
x=df.drop("Loan_Status_Y",axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [13]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cols=["Loan_Amount_Term","LoanAmount","TotalIncome","Credit_History"]
scaler.fit(x_train[cols])


RobustScaler()

In [14]:
x_train=scaler.fit_transform(x_train[cols])
x_test=scaler.fit_transform(x_test[cols])

In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [16]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(x_train, y_train)

print_score(tree_clf, x_train, y_train, x_test, y_test, train=True)
print_score(tree_clf, x_train, y_train, x_test, y_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
           False   True  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    123.0  271.0       1.0      394.0         394.0
_______________________________________________
Confusion Matrix: 
 [[123   0]
 [  0 271]]

Test Result:
Accuracy Score: 77.06%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.622222    0.824000  0.770588    0.723111      0.764654
recall      0.560000    0.858333  0.770588    0.709167      0.770588
f1-score    0.589474    0.840816  0.770588    0.715145      0.766892
support    50.000000  120.000000  0.770588  170.000000    170.000000
_______________________________________________
Conf

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "criterion":("gini", "entropy"),
    "splitter":("best", "random"),
    "max_depth":(list(range(1, 20))),
    "min_samples_split":[2, 3, 4],
    "min_samples_leaf":list(range(1, 20)),
}


tree_clf = DecisionTreeClassifier(random_state=42)
tree_cv = GridSearchCV(tree_clf, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=3)
tree_cv.fit(x_train, y_train)
best_params = tree_cv.best_params_
print(f"Best paramters: {best_params})")

tree_clf_cv = DecisionTreeClassifier(**best_params)
tree_clf_cv.fit(x_train, y_train)
print_score(tree_clf_cv, x_train, y_train, x_test, y_test, train=True)
print_score(tree_clf_cv, x_train, y_train, x_test, y_test, train=False)

Fitting 3 folds for each of 4332 candidates, totalling 12996 fits
Best paramters: {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 4, 'splitter': 'random'})
Train Result:
Accuracy Score: 81.22%
_______________________________________________
CLASSIFICATION REPORT:
                False        True  accuracy   macro avg  weighted avg
precision    0.901639    0.795796  0.812183    0.848718      0.828838
recall       0.447154    0.977860  0.812183    0.712507      0.812183
f1-score     0.597826    0.877483  0.812183    0.737655      0.790179
support    123.000000  271.000000  0.812183  394.000000    394.000000
_______________________________________________
Confusion Matrix: 
 [[ 55  68]
 [  6 265]]

Test Result:
Accuracy Score: 84.71%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.928571    0.830986  0.847059    0.879779      0.859688
recall      0.5

In [18]:
from sklearn.linear_model import LogisticRegression

log=LogisticRegression()
reg=log.fit(x_train,y_train)
print_score(reg, x_train, y_train, x_test, y_test, train=True)
print_score(reg, x_train, y_train, x_test, y_test, train=False)

Train Result:
Accuracy Score: 80.71%
_______________________________________________
CLASSIFICATION REPORT:
                False        True  accuracy   macro avg  weighted avg
precision    0.927273    0.787611  0.807107    0.857442      0.831211
recall       0.414634    0.985240  0.807107    0.699937      0.807107
f1-score     0.573034    0.875410  0.807107    0.724222      0.781013
support    123.000000  271.000000  0.807107  394.000000    394.000000
_______________________________________________
Confusion Matrix: 
 [[ 51  72]
 [  4 267]]

Test Result:
Accuracy Score: 83.53%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.923077    0.819444  0.835294    0.871261      0.849925
recall      0.480000    0.983333  0.835294    0.731667      0.835294
f1-score    0.631579    0.893939  0.835294    0.762759      0.816775
support    50.000000  120.000000  0.835294  170.000000    170.00000

In [19]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ["saga", "lbfgs", "liblinear"],   
}

grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy')
grid.fit(x_train, y_train)

print_score(grid, x_train, y_train, x_test, y_test, train=True)
print_score(grid, x_train, y_train, x_test, y_test, train=False)

c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached 

Train Result:
Accuracy Score: 80.71%
_______________________________________________
CLASSIFICATION REPORT:
                False        True  accuracy   macro avg  weighted avg
precision    0.927273    0.787611  0.807107    0.857442      0.831211
recall       0.414634    0.985240  0.807107    0.699937      0.807107
f1-score     0.573034    0.875410  0.807107    0.724222      0.781013
support    123.000000  271.000000  0.807107  394.000000    394.000000
_______________________________________________
Confusion Matrix: 
 [[ 51  72]
 [  4 267]]

Test Result:
Accuracy Score: 83.53%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.923077    0.819444  0.835294    0.871261      0.849925
recall      0.480000    0.983333  0.835294    0.731667      0.835294
f1-score    0.631579    0.893939  0.835294    0.762759      0.816775
support    50.000000  120.000000  0.835294  170.000000    170.00000

c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
30 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1193, in fit
    solver = _check_s

In [20]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)

print_score(rf, x_train, y_train, x_test, y_test, train=True)
print_score(rf, x_train, y_train, x_test, y_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
           False   True  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    123.0  271.0       1.0      394.0         394.0
_______________________________________________
Confusion Matrix: 
 [[123   0]
 [  0 271]]

Test Result:
Accuracy Score: 83.53%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.823529    0.838235  0.835294    0.830882      0.833910
recall      0.560000    0.950000  0.835294    0.755000      0.835294
f1-score    0.666667    0.890625  0.835294    0.778646      0.824755
support    50.000000  120.000000  0.835294  170.000000    170.000000
_______________________________________________
Conf

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}

rf_clf = RandomForestClassifier(random_state=42)

rf_cv = RandomizedSearchCV(estimator=rf_clf, scoring='f1',param_distributions=random_grid, n_iter=100, cv=3,
                               verbose=2, random_state=42, n_jobs=-1)

rf_cv.fit(x_train, y_train)
rf_best_params = rf_cv.best_params_
print(f"Best paramters: {rf_best_params})")

rf_clf = RandomForestClassifier(**rf_best_params)
rf_clf.fit(x_train, y_train)

print_score(rf_clf, x_train, y_train, x_test, y_test, train=True)
print_score(rf_clf, x_train, y_train, x_test, y_test, train=False)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best paramters: {'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 30, 'bootstrap': True})
Train Result:
Accuracy Score: 84.01%
_______________________________________________
CLASSIFICATION REPORT:
                False        True  accuracy   macro avg  weighted avg
precision    0.905405    0.825000  0.840102    0.865203      0.850101
recall       0.544715    0.974170  0.840102    0.759443      0.840102
f1-score     0.680203    0.893401  0.840102    0.786802      0.826844
support    123.000000  271.000000  0.840102  394.000000    394.000000
_______________________________________________
Confusion Matrix: 
 [[ 67  56]
 [  7 264]]

Test Result:
Accuracy Score: 83.53%
_______________________________________________
CLASSIFICATION REPORT:
               False        True  accuracy   macro avg  weighted avg
precision   0.892857    0.823944  0.835294    0.858400      0.844212
recall      0.500000

In [22]:
acc_list={}
models={grid:"Logistic Reg_CV",reg:"Logistic Reg",tree_clf:"DecisionTree",tree_clf_cv:"DecsiosnTree_CV",rf:"RandomForest",rf_clf:"RandomForest_CV"}
for model in models.keys():
    model.fit(x_train, y_train)
    y_pred=model.predict(x_test)
    acc=accuracy_score(y_test,y_pred)
    acc_list[models[model]]=acc
acc_list

c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\LOQ\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached 

{'Logistic Reg_CV': 0.8352941176470589,
 'Logistic Reg': 0.8352941176470589,
 'DecisionTree': 0.7705882352941177,
 'DecsiosnTree_CV': 0.8352941176470589,
 'RandomForest': 0.8470588235294118,
 'RandomForest_CV': 0.8352941176470589}

In [23]:
best_algo = max(acc_list, key=acc_list.get)
best_score = acc_list[best_algo]

print(f"The best score is: {best_score:.4f} of the algorithm: {best_algo}")

The best score is: 0.8471 of the algorithm: RandomForest
